In [75]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import pickle

In [76]:
train_data = pd.read_csv('train.tsv', sep='\t')
test_data = pd.read_csv('test.tsv', sep='\t')

print(train_data.columns)
print(test_data.columns)

Index(['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'], dtype='object')
Index(['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'], dtype='object')


In [77]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

train_data['combined_text'] = train_data['title'] + ' ' + train_data['text'] + ' ' + train_data['subject'] + ' ' + train_data['date'] 
test_data['combined_text'] = test_data['title'] + ' ' + test_data['text'] + ' ' + test_data['subject'] + ' ' + test_data['date']

label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])
test_data['label'] = label_encoder.transform(test_data['label'])

X_train = train_data['combined_text']
y_train = train_data['label']
X_test = test_data['combined_text']
y_test = test_data['label']

In [78]:
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, min_samples_split=10, class_weight='balanced'),
    'Random Forest': RandomForestClassifier()
}

def evaluate_and_save_model(name, model, X_train, y_train, X_test, y_test):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1] if hasattr(pipeline, 'predict_proba') else None

    with open(f"{name.replace(' ', '_').lower()}_model.pkl", 'wb') as f:
        pickle.dump(pipeline, f)

    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else 'N/A'
    }

    print(f"\n{name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    return y_pred

In [79]:
results = {}
for name, classifier in classifiers.items():
    results[name] = evaluate_and_save_model(name, classifier, X_train, y_train, X_test, y_test)

def save_results(titles, results):
    with open('result.txt', 'w') as f:
        for model_name, predictions in results.items():
            f.write(f"{model_name} Results:\n")
            output = list(zip(titles, predictions))
            f.write(str(output) + '\n\n')

save_results(test_data['title'].tolist(), results)


Logistic Regression Metrics:
Accuracy: 0.9891133422039434
Precision: 0.9874780866516404
Recall: 0.9899573186040673
F1 Score: 0.9887161484453361
ROC-AUC: 0.9988212039355872

Decision Tree Metrics:
Accuracy: 0.9968549655255836
Precision: 0.9964868255959849
Recall: 0.9969871955812202
F1 Score: 0.9967369477911646
ROC-AUC: 0.9983172530875268

Random Forest Metrics:
Accuracy: 0.991169710898754
Precision: 0.9936868686868687
Recall: 0.9879487823248807
F1 Score: 0.9908095178144278
ROC-AUC: 0.9995916644337876
